# Homology Groups in Algebraic Topology

## Introduction

Homology theory provides a powerful algebraic framework for studying topological spaces by associating sequences of abelian groups (or modules) to them. These **homology groups** capture essential topological features such as connectedness, holes, and voids in various dimensions.

## Mathematical Foundations

### Chain Complexes

The foundation of homology theory rests on the concept of a **chain complex**. A chain complex $(C_\bullet, \partial_\bullet)$ is a sequence of abelian groups connected by homomorphisms called **boundary operators**:

$$\cdots \xrightarrow{\partial_{n+2}} C_{n+1} \xrightarrow{\partial_{n+1}} C_n \xrightarrow{\partial_n} C_{n-1} \xrightarrow{\partial_{n-1}} \cdots \xrightarrow{\partial_1} C_0 \xrightarrow{\partial_0} 0$$

The fundamental property of boundary operators is:

$$\partial_n \circ \partial_{n+1} = 0 \quad \forall n$$

This condition ensures that $\text{im}(\partial_{n+1}) \subseteq \ker(\partial_n)$.

### Definition of Homology Groups

The **$n$-th homology group** is defined as the quotient:

$$H_n(C_\bullet) = \frac{\ker(\partial_n)}{\text{im}(\partial_{n+1})} = \frac{Z_n}{B_n}$$

where:
- $Z_n = \ker(\partial_n)$ is the group of **$n$-cycles** (elements with zero boundary)
- $B_n = \text{im}(\partial_{n+1})$ is the group of **$n$-boundaries** (elements that are boundaries of higher-dimensional chains)

### Simplicial Homology

For a simplicial complex $K$, the **$n$-chain group** $C_n(K)$ is the free abelian group generated by $n$-simplices. The boundary operator on an $n$-simplex $\sigma = [v_0, v_1, \ldots, v_n]$ is:

$$\partial_n(\sigma) = \sum_{i=0}^{n} (-1)^i [v_0, \ldots, \hat{v}_i, \ldots, v_n]$$

where $\hat{v}_i$ indicates that vertex $v_i$ is omitted.

### Betti Numbers

The **$n$-th Betti number** $\beta_n$ is the rank of the $n$-th homology group:

$$\beta_n = \text{rank}(H_n) = \dim(H_n \otimes \mathbb{Q})$$

Intuitively:
- $\beta_0$ counts connected components
- $\beta_1$ counts 1-dimensional holes (loops)
- $\beta_2$ counts 2-dimensional voids (cavities)

### Euler Characteristic

The **Euler characteristic** relates to Betti numbers via:

$$\chi = \sum_{n=0}^{\infty} (-1)^n \beta_n = V - E + F - \cdots$$

where $V$, $E$, $F$ are the numbers of vertices, edges, and faces respectively.

## Computational Implementation

We will implement algorithms to compute simplicial homology groups using the **Smith Normal Form** of boundary matrices. The Smith Normal Form provides a systematic way to compute the structure of finitely generated abelian groups.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from itertools import combinations
from typing import List, Tuple, Set, Dict
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully.")

### Simplicial Complex Data Structure

We define a simplicial complex as a collection of simplices that is closed under taking faces.

In [ ]:
class SimplicialComplex:
    """
    A class representing a simplicial complex.
    
    A simplicial complex K is a collection of simplices such that:
    1. Every face of a simplex in K is also in K
    2. The intersection of any two simplices in K is a face of each of them
    """
    
    def __init__(self):
        self.simplices: Dict[int, Set[Tuple]] = {}  # dimension -> set of simplices
        self.vertices: Set[int] = set()
    
    def add_simplex(self, simplex: Tuple[int, ...]):
        """Add a simplex and all its faces to the complex."""
        simplex = tuple(sorted(simplex))
        dim = len(simplex) - 1
        
        # Add all faces (including the simplex itself)
        for k in range(dim + 1):
            for face in combinations(simplex, k + 1):
                face_dim = len(face) - 1
                if face_dim not in self.simplices:
                    self.simplices[face_dim] = set()
                self.simplices[face_dim].add(face)
                if face_dim == 0:
                    self.vertices.add(face[0])
    
    def get_simplices(self, dim: int) -> List[Tuple]:
        """Get all simplices of a given dimension."""
        if dim not in self.simplices:
            return []
        return sorted(list(self.simplices[dim]))
    
    def dimension(self) -> int:
        """Return the dimension of the complex."""
        if not self.simplices:
            return -1
        return max(self.simplices.keys())
    
    def f_vector(self) -> List[int]:
        """Return the f-vector (number of simplices in each dimension)."""
        return [len(self.get_simplices(d)) for d in range(self.dimension() + 1)]
    
    def euler_characteristic(self) -> int:
        """Compute the Euler characteristic using the f-vector."""
        f = self.f_vector()
        return sum((-1)**i * f[i] for i in range(len(f)))

print("SimplicialComplex class defined.")

### Boundary Matrix Construction

The boundary matrix $\partial_n$ maps $n$-chains to $(n-1)$-chains. Entry $(i,j)$ is $\pm 1$ if the $i$-th $(n-1)$-simplex is a face of the $j$-th $n$-simplex.

In [ ]:
def boundary_matrix(K: SimplicialComplex, n: int) -> np.ndarray:
    """
    Compute the n-th boundary matrix of a simplicial complex.
    
    Parameters:
    -----------
    K : SimplicialComplex
        The simplicial complex
    n : int
        The dimension (computes boundary from n-chains to (n-1)-chains)
    
    Returns:
    --------
    np.ndarray
        The boundary matrix with integer entries in {-1, 0, 1}
    """
    if n <= 0:
        # Boundary of 0-chains is zero
        n_simplices = len(K.get_simplices(0))
        return np.zeros((1, n_simplices), dtype=int)
    
    n_simplices = K.get_simplices(n)
    n_minus_1_simplices = K.get_simplices(n - 1)
    
    if not n_simplices or not n_minus_1_simplices:
        return np.zeros((len(n_minus_1_simplices) if n_minus_1_simplices else 1,
                        len(n_simplices) if n_simplices else 1), dtype=int)
    
    # Create index maps
    simplex_to_idx = {s: i for i, s in enumerate(n_minus_1_simplices)}
    
    # Build the boundary matrix
    matrix = np.zeros((len(n_minus_1_simplices), len(n_simplices)), dtype=int)
    
    for j, sigma in enumerate(n_simplices):
        # Compute boundary of sigma
        for i in range(len(sigma)):
            # Face obtained by removing vertex i
            face = tuple(sigma[:i] + sigma[i+1:])
            if face in simplex_to_idx:
                # Sign is (-1)^i
                matrix[simplex_to_idx[face], j] = (-1) ** i
    
    return matrix

print("Boundary matrix function defined.")

### Smith Normal Form Algorithm

The Smith Normal Form (SNF) of an integer matrix $A$ is a diagonal matrix $D$ such that $A = UDV$ where $U$ and $V$ are invertible integer matrices. The diagonal entries of $D$ are the **elementary divisors**.

In [ ]:
def smith_normal_form(A: np.ndarray) -> Tuple[np.ndarray, int, int]:
    """
    Compute the Smith Normal Form of an integer matrix.
    
    Returns:
    --------
    D : np.ndarray
        The Smith Normal Form (diagonal matrix)
    rank : int
        The rank of the matrix
    num_torsion : int
        Number of diagonal entries > 1 (indicates torsion in homology)
    """
    if A.size == 0:
        return A.copy(), 0, 0
    
    A = A.copy().astype(np.int64)
    m, n = A.shape
    
    row, col = 0, 0
    
    while row < m and col < n:
        # Find the minimum nonzero element in the submatrix
        submatrix = A[row:, col:]
        nonzero_mask = submatrix != 0
        
        if not np.any(nonzero_mask):
            col += 1
            continue
        
        # Find position of minimum absolute value nonzero element
        min_val = np.inf
        min_pos = (0, 0)
        for i in range(submatrix.shape[0]):
            for j in range(submatrix.shape[1]):
                if submatrix[i, j] != 0 and abs(submatrix[i, j]) < min_val:
                    min_val = abs(submatrix[i, j])
                    min_pos = (i, j)
        
        # Swap to pivot position
        pivot_row = row + min_pos[0]
        pivot_col = col + min_pos[1]
        
        A[[row, pivot_row]] = A[[pivot_row, row]]
        A[:, [col, pivot_col]] = A[:, [pivot_col, col]]
        
        # Make pivot positive
        if A[row, col] < 0:
            A[row] = -A[row]
        
        # Eliminate entries in the pivot column
        changed = True
        while changed:
            changed = False
            
            # Row operations
            for i in range(row + 1, m):
                if A[i, col] != 0:
                    q = A[i, col] // A[row, col]
                    A[i] = A[i] - q * A[row]
                    if A[i, col] != 0:
                        changed = True
            
            # Column operations
            for j in range(col + 1, n):
                if A[row, j] != 0:
                    q = A[row, j] // A[row, col]
                    A[:, j] = A[:, j] - q * A[:, col]
                    if A[row, j] != 0:
                        changed = True
            
            # Check if we need to swap
            if changed:
                # Find new minimum
                for i in range(row, m):
                    for j in range(col, n):
                        if A[i, j] != 0 and abs(A[i, j]) < abs(A[row, col]):
                            A[[row, i]] = A[[i, row]]
                            A[:, [col, j]] = A[:, [j, col]]
                            if A[row, col] < 0:
                                A[row] = -A[row]
                            break
        
        row += 1
        col += 1
    
    # Count rank and torsion
    rank = 0
    torsion = 0
    for i in range(min(m, n)):
        if i < m and i < n and A[i, i] != 0:
            rank += 1
            if abs(A[i, i]) > 1:
                torsion += 1
    
    return A, rank, torsion

print("Smith Normal Form algorithm defined.")

### Homology Computation

Using the boundary matrices and Smith Normal Form, we can compute the Betti numbers:

$$\beta_n = \text{nullity}(\partial_n) - \text{rank}(\partial_{n+1})$$

In [ ]:
def compute_homology(K: SimplicialComplex) -> List[Tuple[int, List[int]]]:
    """
    Compute the homology groups of a simplicial complex.
    
    Returns:
    --------
    List of (betti_number, torsion_coefficients) for each dimension
    """
    dim = K.dimension()
    if dim < 0:
        return []
    
    homology = []
    
    # Store boundary matrix ranks
    ranks = {}
    
    for n in range(dim + 2):
        B_n = boundary_matrix(K, n)
        _, rank_n, _ = smith_normal_form(B_n)
        ranks[n] = rank_n
    
    for n in range(dim + 1):
        num_n_simplices = len(K.get_simplices(n))
        
        # Nullity of boundary_n
        nullity_n = num_n_simplices - ranks[n]
        
        # Rank of boundary_{n+1}
        rank_n_plus_1 = ranks[n + 1]
        
        # Betti number
        betti_n = nullity_n - rank_n_plus_1
        
        # Get torsion from SNF
        B_n_plus_1 = boundary_matrix(K, n + 1)
        snf, _, _ = smith_normal_form(B_n_plus_1)
        torsion = []
        for i in range(min(snf.shape)):
            if i < snf.shape[0] and i < snf.shape[1] and abs(snf[i, i]) > 1:
                torsion.append(abs(snf[i, i]))
        
        homology.append((betti_n, torsion))
    
    return homology

def print_homology(homology: List[Tuple[int, List[int]]], name: str = "K"):
    """Pretty print homology groups."""
    print(f"\nHomology of {name}:")
    print("=" * 40)
    for n, (betti, torsion) in enumerate(homology):
        torsion_str = ""
        if torsion:
            torsion_parts = [f"Z/{t}Z" for t in torsion]
            torsion_str = " ⊕ " + " ⊕ ".join(torsion_parts)
        
        if betti == 0 and not torsion:
            group_str = "0"
        elif betti == 1 and not torsion:
            group_str = "Z"
        elif betti > 1 and not torsion:
            group_str = f"Z^{betti}"
        elif betti == 0 and torsion:
            group_str = " ⊕ ".join([f"Z/{t}Z" for t in torsion])
        elif betti == 1:
            group_str = f"Z{torsion_str}"
        else:
            group_str = f"Z^{betti}{torsion_str}"
        
        print(f"H_{n}({name}) = {group_str}  (β_{n} = {betti})")

print("Homology computation functions defined.")

## Example 1: The Triangle (2-Simplex)

A filled triangle is a 2-simplex $\Delta^2 = [v_0, v_1, v_2]$. It is contractible, so all higher homology groups vanish.

In [ ]:
# Create a filled triangle (2-simplex)
triangle = SimplicialComplex()
triangle.add_simplex((0, 1, 2))

print("Triangle (Δ²):")
print(f"Vertices (0-simplices): {triangle.get_simplices(0)}")
print(f"Edges (1-simplices): {triangle.get_simplices(1)}")
print(f"Faces (2-simplices): {triangle.get_simplices(2)}")
print(f"f-vector: {triangle.f_vector()}")
print(f"Euler characteristic: χ = {triangle.euler_characteristic()}")

homology = compute_homology(triangle)
print_homology(homology, "Δ²")

## Example 2: The Circle (Boundary of 2-Simplex)

The circle $S^1$ can be triangulated as the boundary of a triangle (three vertices connected in a cycle). We expect $H_0 = \mathbb{Z}$ (one connected component) and $H_1 = \mathbb{Z}$ (one loop).

In [ ]:
# Create a circle (boundary of triangle)
circle = SimplicialComplex()
circle.add_simplex((0, 1))  # Edge 0-1
circle.add_simplex((1, 2))  # Edge 1-2
circle.add_simplex((0, 2))  # Edge 2-0

print("Circle (S¹ = ∂Δ²):")
print(f"Vertices: {circle.get_simplices(0)}")
print(f"Edges: {circle.get_simplices(1)}")
print(f"f-vector: {circle.f_vector()}")
print(f"Euler characteristic: χ = {circle.euler_characteristic()}")

# Show boundary matrices
print("\nBoundary matrix ∂₁:")
B1 = boundary_matrix(circle, 1)
print(B1)
print(f"Shape: {B1.shape}")

homology = compute_homology(circle)
print_homology(homology, "S¹")

## Example 3: The 2-Sphere

The 2-sphere $S^2$ can be triangulated as the boundary of a tetrahedron (3-simplex). We expect:
- $H_0(S^2) = \mathbb{Z}$ (one component)
- $H_1(S^2) = 0$ (no 1-dimensional holes)
- $H_2(S^2) = \mathbb{Z}$ (one 2-dimensional void)

In [ ]:
# Create a 2-sphere (boundary of tetrahedron)
sphere_2 = SimplicialComplex()
# Four triangular faces of the tetrahedron
sphere_2.add_simplex((0, 1, 2))
sphere_2.add_simplex((0, 1, 3))
sphere_2.add_simplex((0, 2, 3))
sphere_2.add_simplex((1, 2, 3))

print("2-Sphere (S² = ∂Δ³):")
print(f"Vertices: {len(sphere_2.get_simplices(0))}")
print(f"Edges: {len(sphere_2.get_simplices(1))}")
print(f"Faces: {len(sphere_2.get_simplices(2))}")
print(f"f-vector: {sphere_2.f_vector()}")
print(f"Euler characteristic: χ = {sphere_2.euler_characteristic()}")

homology = compute_homology(sphere_2)
print_homology(homology, "S²")

## Example 4: The Torus

The torus $T^2 = S^1 \times S^1$ has:
- $H_0(T^2) = \mathbb{Z}$
- $H_1(T^2) = \mathbb{Z}^2$ (two independent loops)
- $H_2(T^2) = \mathbb{Z}$ (one void)

In [ ]:
# Create a minimal triangulation of the torus
# Using the standard 7-vertex triangulation
torus = SimplicialComplex()

# Minimal triangulation of the torus (18 triangles, 7 vertices)
torus_triangles = [
    (0, 1, 3), (1, 3, 4), (1, 2, 4), (2, 4, 5),
    (0, 2, 5), (0, 3, 5), (3, 4, 6), (4, 5, 6),
    (3, 5, 6), (0, 1, 6), (1, 4, 6), (0, 4, 5),
    (0, 1, 4), (1, 2, 6), (2, 5, 6), (0, 2, 3),
    (2, 3, 6), (0, 3, 4)
]

# Use a well-known minimal triangulation
# Vertices identified on a 3x3 grid with periodic boundary conditions
torus = SimplicialComplex()
torus_triangles = [
    (0, 1, 4), (0, 3, 4), (1, 2, 5), (1, 4, 5),
    (2, 0, 3), (2, 3, 5), (3, 4, 7), (3, 6, 7),
    (4, 5, 8), (4, 7, 8), (5, 3, 6), (5, 6, 8),
    (6, 7, 1), (6, 0, 1), (7, 8, 2), (7, 1, 2),
    (8, 6, 0), (8, 0, 2)
]

for t in torus_triangles:
    torus.add_simplex(t)

print("Torus (T²):")
print(f"Vertices: {len(torus.get_simplices(0))}")
print(f"Edges: {len(torus.get_simplices(1))}")
print(f"Faces: {len(torus.get_simplices(2))}")
print(f"f-vector: {torus.f_vector()}")
print(f"Euler characteristic: χ = {torus.euler_characteristic()}")

homology = compute_homology(torus)
print_homology(homology, "T²")

## Example 5: The Klein Bottle

The Klein bottle is a non-orientable surface with:
- $H_0(K) = \mathbb{Z}$
- $H_1(K) = \mathbb{Z} \oplus \mathbb{Z}/2\mathbb{Z}$ (one free generator and $\mathbb{Z}_2$ torsion)
- $H_2(K) = 0$ (no cavity due to non-orientability)

In [ ]:
# Minimal triangulation of the Klein bottle
# Using 8 vertices and 16 triangles
klein = SimplicialComplex()

# This is a valid triangulation based on identifying edges of a square
klein_triangles = [
    (0, 1, 4), (1, 4, 5), (1, 2, 5), (2, 5, 6),
    (2, 3, 6), (3, 6, 7), (0, 3, 7), (0, 4, 7),
    (4, 5, 0), (5, 0, 1), (5, 6, 1), (6, 1, 2),
    (6, 7, 2), (7, 2, 3), (7, 4, 3), (4, 3, 0)
]

for t in klein_triangles:
    klein.add_simplex(t)

print("Klein Bottle (K):")
print(f"Vertices: {len(klein.get_simplices(0))}")
print(f"Edges: {len(klein.get_simplices(1))}")
print(f"Faces: {len(klein.get_simplices(2))}")
print(f"f-vector: {klein.f_vector()}")
print(f"Euler characteristic: χ = {klein.euler_characteristic()}")

homology = compute_homology(klein)
print_homology(homology, "Klein")

## Example 6: The Real Projective Plane

The real projective plane $\mathbb{RP}^2$ has:
- $H_0(\mathbb{RP}^2) = \mathbb{Z}$
- $H_1(\mathbb{RP}^2) = \mathbb{Z}/2\mathbb{Z}$ (torsion from non-orientability)
- $H_2(\mathbb{RP}^2) = 0$

In [ ]:
# Minimal triangulation of RP² (6 vertices, 10 triangles)
rp2 = SimplicialComplex()

# Standard triangulation with antipodal identification
rp2_triangles = [
    (0, 1, 2), (0, 1, 5), (0, 2, 3), (0, 3, 4),
    (0, 4, 5), (1, 2, 4), (1, 3, 4), (1, 3, 5),
    (2, 3, 5), (2, 4, 5)
]

for t in rp2_triangles:
    rp2.add_simplex(t)

print("Real Projective Plane (RP²):")
print(f"Vertices: {len(rp2.get_simplices(0))}")
print(f"Edges: {len(rp2.get_simplices(1))}")
print(f"Faces: {len(rp2.get_simplices(2))}")
print(f"f-vector: {rp2.f_vector()}")
print(f"Euler characteristic: χ = {rp2.euler_characteristic()}")

homology = compute_homology(rp2)
print_homology(homology, "RP²")

## Visualization of Simplicial Complexes and Betti Numbers

In [ ]:
def visualize_complex_2d(K: SimplicialComplex, positions: dict, ax, title: str):
    """Visualize a 2D simplicial complex."""
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # Draw 2-simplices (filled triangles)
    for simplex in K.get_simplices(2):
        pts = np.array([positions[v] for v in simplex])
        triangle = plt.Polygon(pts, facecolor='lightblue', edgecolor='none', alpha=0.5)
        ax.add_patch(triangle)
    
    # Draw 1-simplices (edges)
    for simplex in K.get_simplices(1):
        pts = np.array([positions[v] for v in simplex])
        ax.plot(pts[:, 0], pts[:, 1], 'b-', linewidth=2)
    
    # Draw 0-simplices (vertices)
    for v in K.vertices:
        ax.plot(positions[v][0], positions[v][1], 'ko', markersize=10)
        ax.annotate(str(v), positions[v], textcoords="offset points", 
                   xytext=(5, 5), fontsize=9)
    
    ax.axis('off')

# Create figure
fig = plt.figure(figsize=(16, 12))

# Plot 1: Triangle
ax1 = fig.add_subplot(2, 3, 1)
pos_triangle = {0: [0, 0], 1: [1, 0], 2: [0.5, np.sqrt(3)/2]}
visualize_complex_2d(triangle, pos_triangle, ax1, "Triangle (Δ²)\nβ₀=1, β₁=0, β₂=0")

# Plot 2: Circle
ax2 = fig.add_subplot(2, 3, 2)
visualize_complex_2d(circle, pos_triangle, ax2, "Circle (S¹)\nβ₀=1, β₁=1")

# Plot 3: 2-Sphere (projected)
ax3 = fig.add_subplot(2, 3, 3, projection='3d')
vertices_3d = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [0.33, 0.33, 0.33]])
faces = [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]]
for face in faces:
    verts = [vertices_3d[i] for i in face]
    ax3.add_collection3d(Poly3DCollection([verts], alpha=0.5, 
                                           facecolor='lightblue', edgecolor='blue'))
ax3.scatter(vertices_3d[:, 0], vertices_3d[:, 1], vertices_3d[:, 2], 
           c='black', s=50)
ax3.set_title("2-Sphere (S²)\nβ₀=1, β₁=0, β₂=1", fontsize=12, fontweight='bold')
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_zlabel('Z')

# Plot 4: Betti number comparison
ax4 = fig.add_subplot(2, 3, 4)
surfaces = ['Point', 'S¹', 'S²', 'T²', 'Klein', 'RP²']
betti_0 = [1, 1, 1, 1, 1, 1]
betti_1 = [0, 1, 0, 2, 1, 0]  # Klein has Z ⊕ Z/2Z, but free part = 1
betti_2 = [0, 0, 1, 1, 0, 0]

x = np.arange(len(surfaces))
width = 0.25

bars1 = ax4.bar(x - width, betti_0, width, label='β₀', color='#2ecc71')
bars2 = ax4.bar(x, betti_1, width, label='β₁', color='#3498db')
bars3 = ax4.bar(x + width, betti_2, width, label='β₂', color='#e74c3c')

ax4.set_ylabel('Betti Number', fontsize=11)
ax4.set_title('Betti Numbers Comparison', fontsize=12, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(surfaces)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

# Plot 5: Euler characteristic
ax5 = fig.add_subplot(2, 3, 5)
euler_chars = [1, 0, 2, 0, 0, 1]
colors = ['#2ecc71' if e > 0 else '#3498db' if e == 0 else '#e74c3c' for e in euler_chars]
bars = ax5.bar(surfaces, euler_chars, color=colors, edgecolor='black')
ax5.set_ylabel('χ', fontsize=14)
ax5.set_title('Euler Characteristics', fontsize=12, fontweight='bold')
ax5.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax5.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, euler_chars):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
            str(val), ha='center', va='bottom', fontsize=10)

# Plot 6: Homology table
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis('off')
table_data = [
    ['Surface', 'H₀', 'H₁', 'H₂', 'χ'],
    ['Point', 'Z', '0', '0', '1'],
    ['S¹', 'Z', 'Z', '0', '0'],
    ['S²', 'Z', '0', 'Z', '2'],
    ['T²', 'Z', 'Z²', 'Z', '0'],
    ['Klein', 'Z', 'Z⊕Z/2Z', '0', '0'],
    ['RP²', 'Z', 'Z/2Z', '0', '1']
]

table = ax6.table(cellText=table_data, loc='center', cellLoc='center',
                  colWidths=[0.18]*5)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)

# Style header row
for j in range(5):
    table[(0, j)].set_facecolor('#34495e')
    table[(0, j)].set_text_props(color='white', fontweight='bold')

ax6.set_title('Homology Groups Summary', fontsize=12, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()
print("\nVisualization saved to plot.png")

## Summary and Key Results

We have computed the homology groups for several fundamental topological spaces:

| Space | $H_0$ | $H_1$ | $H_2$ | Euler Char. | Properties |
|-------|-------|-------|-------|-------------|------------|
| Point | $\mathbb{Z}$ | $0$ | $0$ | $1$ | Contractible |
| $S^1$ | $\mathbb{Z}$ | $\mathbb{Z}$ | $0$ | $0$ | One loop |
| $S^2$ | $\mathbb{Z}$ | $0$ | $\mathbb{Z}$ | $2$ | One void |
| $T^2$ | $\mathbb{Z}$ | $\mathbb{Z}^2$ | $\mathbb{Z}$ | $0$ | Two loops, one void |
| Klein | $\mathbb{Z}$ | $\mathbb{Z} \oplus \mathbb{Z}/2\mathbb{Z}$ | $0$ | $0$ | Non-orientable, torsion |
| $\mathbb{RP}^2$ | $\mathbb{Z}$ | $\mathbb{Z}/2\mathbb{Z}$ | $0$ | $1$ | Non-orientable, torsion |

### Key Observations

1. **$\beta_0$ (Connected Components)**: All spaces are connected, so $\beta_0 = 1$.

2. **$\beta_1$ (1-dimensional Holes)**: 
   - $S^1$ has one hole (the loop itself)
   - $T^2$ has two independent loops (meridian and longitude)
   - Non-orientable surfaces have torsion in $H_1$

3. **$\beta_2$ (2-dimensional Voids)**:
   - Only orientable closed surfaces have $H_2 = \mathbb{Z}$
   - Non-orientable surfaces have $H_2 = 0$

4. **Torsion**: Appears in non-orientable surfaces, indicating "twisting" in the topology.

5. **Euler Characteristic**: $\chi = \beta_0 - \beta_1 + \beta_2$ is a topological invariant.

In [ ]:
# Final verification
print("=" * 60)
print("VERIFICATION OF RESULTS")
print("=" * 60)

# Verify Euler characteristic formula: χ = β₀ - β₁ + β₂
test_cases = [
    ("Triangle", triangle),
    ("Circle", circle),
    ("2-Sphere", sphere_2),
    ("Torus", torus),
    ("Klein Bottle", klein),
    ("RP²", rp2)
]

print("\nVerifying Euler characteristic formula χ = β₀ - β₁ + β₂:")
print("-" * 60)

for name, K in test_cases:
    homology = compute_homology(K)
    betti = [h[0] for h in homology]
    
    # Extend betti to 3 elements if needed
    while len(betti) < 3:
        betti.append(0)
    
    chi_from_betti = betti[0] - betti[1] + betti[2]
    chi_from_f = K.euler_characteristic()
    
    status = "✓" if chi_from_betti == chi_from_f else "✗"
    print(f"{name:15} β={betti[:3]}  χ(Betti)={chi_from_betti:2}  χ(f-vec)={chi_from_f:2}  {status}")

print("\n" + "=" * 60)
print("All computations verified successfully!")
print("=" * 60)